In [1]:
from gtts import gTTS
from IPython.display import Audio, display, HTML
import os

def text_to_speech(text, filename="summary_audio.mp3"):
    tts = gTTS(text=text, lang='en', slow=False)
    tts.save(filename)
    return filename

gTTS (Google Text-to-Speech) converts text into spoken audio using Google's TTS API.

text_to_speech() takes a string and generates an .mp3 file using that text.

The function:

    Creates the speech with gTTS(text=text, lang='en').

    Saves it to a file.

    Returns the file name for playback.

In [2]:
from transformers import BartTokenizer, BartForConditionalGeneration
import torch
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
import io
import PyPDF2
import docx
import time


ModuleNotFoundError: No module named 'torch'

This cell installs required Python libraries for text summarization, file handling, and interactive UI. Then it imports tools from those libraries to load models, build widgets, and read PDF/DOCX files in Colab.

In [ ]:
loading_msg = widgets.Label(value="Loading summarization model, please wait...")
display(loading_msg)

tokenizer = BartTokenizer.from_pretrained('facebook/bart-large-cnn')
model = BartForConditionalGeneration.from_pretrained('facebook/bart-large-cnn')

loading_msg.value = "Model loaded! Ready to summarize"
time.sleep(1)
loading_msg.close()


Label(value='Loading summarization model, please wait...')

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

ImportError: 
BartForConditionalGeneration requires the PyTorch library but it was not found in your environment.
However, we were able to find a TensorFlow installation. TensorFlow classes begin
with "TF", but are otherwise identically named to our PyTorch classes. This
means that the TF equivalent of the class you tried to import would be "TFBartForConditionalGeneration".
If you want to use TensorFlow, please use TF classes instead!

If you really do want to use PyTorch please go to
https://pytorch.org/get-started/locally/ and follow the instructions that
match your environment.


**Cell Summary**

1. Show a “Loading…” label so the user knows the model is being prepared.
2. Load `BartTokenizer` to turn raw text into the token IDs a model needs.
3. Load `BartForConditionalGeneration` (the pre-trained BART-Large-CNN model) that will actually create summaries.
4. Update the label to “Model loaded!”, pause one second so the user can see it, then remove the label.

---

### What the tokenizer is for

* A **tokenizer** converts human text into integers (“tokens”) that the neural network understands, and later turns the model’s output tokens back into readable text.

### What **`BartTokenizer`** is

* `BartTokenizer` is the specific tokenizer paired with Facebook’s BART model; it knows exactly how BART expects sentences to be split into sub-word pieces and numbered, ensuring the text and model stay in sync.


In [ ]:
def summarize(text, max_length=150, min_length=60):
    inputs = tokenizer.encode(text, return_tensors="pt", max_length=1024, truncation=True)
    summary_ids = model.generate(
        inputs,
        max_length=max_length,
        min_length=min_length,
        length_penalty=2.0,
        num_beams=4,
        early_stopping=True
    )
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return summary

def extract_text_from_pdf(file_bytes):
    reader = PyPDF2.PdfReader(io.BytesIO(file_bytes))
    text = ""
    for page in reader.pages:
        text += page.extract_text() + "\n"
    return text.strip()

def extract_text_from_docx(file_bytes):
    doc = docx.Document(io.BytesIO(file_bytes))
    fullText = []
    for para in doc.paragraphs:
        fullText.append(para.text)
    return "\n".join(fullText).strip()



### 1. `summarize(text, max_length=150, min_length=60)`

* **Purpose:** This function takes a block of text and generates a concise summary using the BART model.

* **Detail explanation:**

  * `tokenizer.encode(...)`:
    Converts the input `text` into a sequence of token IDs (numbers) that the model understands.

    * `return_tensors="pt"` means the tokens are returned as a PyTorch tensor (required for the model).
    * `max_length=1024` limits the input to 1024 tokens, truncating if longer.

  * `model.generate(...)`:
    Generates a summary by predicting tokens step-by-step, constrained by parameters:

    * `max_length` and `min_length` control the length of the summary.
    * `length_penalty=2.0` encourages shorter summaries (higher penalty for longer outputs).
    * `num_beams=4` enables beam search with 4 beams, making generation more accurate by exploring multiple options.
    * `early_stopping=True` stops the generation once an end condition is met to save time.

  * `tokenizer.decode(...)`:
    Converts the generated token IDs back into human-readable text, skipping special tokens like `<s>`, `</s>`, etc.

  * **Returns:** The final summarized text as a string.

---

### 2. `extract_text_from_pdf(file_bytes)`

* **Purpose:** Extracts all the readable text content from a PDF file, given the file’s bytes.

* **Step-by-step:**

  * `PyPDF2.PdfReader(io.BytesIO(file_bytes))`:
    Loads the PDF file from its raw byte data (not a saved file) into a PDF reader object.

  * `for page in reader.pages`:
    Loops through each page in the PDF.

  * `page.extract_text()`:
    Extracts the plain text from the current page.

  * Concatenates the text from all pages into a single string separated by newline characters.

  * `strip()`:
    Removes any leading or trailing whitespace/newlines from the combined text.

  * **Returns:** The entire extracted text content as a single string.

---

### 3. `extract_text_from_docx(file_bytes)`

* **Purpose:** Extracts all the text from a Microsoft Word `.docx` file, given the file’s bytes.

 does thesame thing as the pdf section
---



In [ ]:
# File uploader with multiple file support and nicer description
upload = widgets.FileUpload(
    accept='.pdf,.docx,.txt',
    multiple=False,
    description='Upload File',
    style={'button_color': '#4CAF50'}
)

text_input = widgets.Textarea(
    value='',
    placeholder='Or paste your text here to summarize...',
    description='Input Text:',
    layout=widgets.Layout(width='100%', height='180px'),
    style={'description_width': 'initial'}
)

# Max and Min length sliders for summary length
max_len_slider = widgets.IntSlider(
    value=150,
    min=50,
    max=300,
    step=10,
    description='Max Summary Length:',
    style={'description_width': 'initial'},
    continuous_update=False,
    layout=widgets.Layout(width='80%')
)

min_len_slider = widgets.IntSlider(
    value=60,
    min=20,
    max=150,
    step=5,
    description='Min Summary Length:',
    style={'description_width': 'initial'},
    continuous_update=False,
    layout=widgets.Layout(width='80%')
)


button = widgets.Button(
    description="Summarize",
    button_style='success',
    tooltip='Click to summarize text',
    icon='magic'
)


output = widgets.Output(layout={'border': '1px solid #ccc', 'padding': '10px', 'border_radius': '5px'})

# Status label for info messages
status = widgets.Label(value="")


Above cell simply accepts file formats and style some sections set min and max length for summarization



In [ ]:
def on_upload_change(change):
    if upload.value:
        uploaded_file = list(upload.value.values())[0]
        content = uploaded_file['content']
        name = uploaded_file['metadata']['name'].lower()

        status.value = f"Extracting text from {name}..."

        try:
            if name.endswith('.pdf'):
                extracted_text = extract_text_from_pdf(content)
            elif name.endswith('.docx'):
                extracted_text = extract_text_from_docx(content)
            elif name.endswith('.txt'):
                extracted_text = content.decode('utf-8')
            else:
                extracted_text = "Unsupported file type."
        except Exception as e:
            extracted_text = f"Error extracting text: {str(e)}"

        text_input.value = extracted_text
        status.value = f"Text extracted from {name}. Ready to summarize!"

def on_button_clicked(b):
    with output:
        clear_output()
        if not text_input.value.strip():
            status.value = "Please provide some text or upload a file."
            return

        max_len = max_len_slider.value
        min_len = min_len_slider.value

        if min_len >= max_len:
            status.value = "Minimum length must be less than maximum length."
            return

        status.value = "Summarizing... Please wait."

        try:
            summary = summarize(text_input.value, max_length=max_len, min_length=min_len)

            # Save summary audio
            audio_filename = text_to_speech(summary)

            # Display result with audio icon
            display(HTML(f"""
                <h4>Summary:</h4>
                <p style='white-space: pre-line; font-size:16px; background:#f9f9f9; padding:10px; border-radius:5px;'>
                    {summary}
                </p>
                <p><b>🔊 Listen:</b></p>
            """))
            display(Audio(audio_filename, autoplay=False))

            status.value = "Summary generated successfully!"
        except Exception as e:
            status.value = f"Error during summarization: {str(e)}"

upload.observe(on_upload_change, names='value')
button.on_click(on_button_clicked)





```python
upload.observe(on_upload_change, names='value')
```

* This tells the system: **"When a user uploads a file, call the `on_upload_change` function."**


---



```python
button.on_click(on_button_clicked)
```

* **"When the user clicks the Summarize button, run `on_button_clicked()`."**
* It's what triggers the summarization and speech generation process.

---

###  3. Handling File Uploads

```python
def on_upload_change(change):
```

* This function runs **automatically when a file is uploaded**.
* It pulls out the file’s name and content to prepare it for text extraction.

#### Inside the function:

```python
uploaded_file = list(upload.value.values())[0]
content = uploaded_file['content']
name = uploaded_file['metadata']['name'].lower()
```

* These lines grab the uploaded file's data: the content (bytes) and file name.
* The `lower()` ensures case-insensitive checks for `.PDF`, `.pdf`, etc.

```python
if name.endswith('.pdf'):
    extracted_text = extract_text_from_pdf(content)
elif name.endswith('.docx'):
    extracted_text = extract_text_from_docx(content)
elif name.endswith('.txt'):
    extracted_text = content.decode('utf-8')
```

* Depending on the file type, the function chooses how to extract the text.
* `.pdf` files are read using `PyPDF2`, `.docx` with `python-docx`, and `.txt` just decoded.

```python
text_input.value = extracted_text
status.value = f"Text extracted from {name}. Ready to summarize!"
```

* After extracting, the text is placed in the input box so the user can see/edit it.
* The status tells the user it's ready.

---

###  4. Handling the Summarize Button

```python
def on_button_clicked(b):
```

* This function runs **when the “Summarize” button is clicked**.

```python
if not text_input.value.strip():
    status.value = "Please provide some text or upload a file."
    return
```

* It checks: if input is empty.

```python
max_len = max_len_slider.value
min_len = min_len_slider.value
if min_len >= max_len:
    status.value = "Minimum length must be less than maximum length."
    return
```

* This ensures the user didn't enter bad settings (like min summary length > max).

```python
summary = summarize(text_input.value, max_length=max_len, min_length=min_len)
```

* This is where the summarization happens! It uses the `summarize()` function powered by BART.

```python
audio_filename = text_to_speech(summary)
```

* Then the summary is passed to Google Text-to-Speech to create an audio file.

```python
display(HTML(...))
display(Audio(audio_filename, autoplay=False))
```

* These lines:

  * Show the summary in a nice, styled box.
  * Add an audio player below it so the user can **listen to the summary**.




In [ ]:
ui = widgets.VBox([
    widgets.HTML("<h2 style='color:#4CAF50;'>Fancy Text Summarizer with File Upload & Controls</h2>"),
    upload,
    text_input,
    widgets.HBox([min_len_slider, max_len_slider]),
    button,
    status,
    output
], layout=widgets.Layout(width='80%', margin='auto'))

display(ui)

#displays the UI
